Согласованность, Точность, Уникальность, Полнота, Своевременность, Доступность и т.д.

Range Constraints: числа или даты должны попадать в определенный диапазон с использованием одной единицы измерения
Mandatory Constraints: некоторые столбцы не могут быть пустыми
Шаблоны регулярных выражений: текстовые поля, которые должны быть в определенном шаблоне. Например (999) 999–9999.
Consistency: степень соответствия данных в одном или нескольких БД

In [ ]:
import os
import csv
import numpy as np
import pandas as pd
import matplotlib 
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from pylab import plot,show,hist
from scipy.stats.kde import gaussian_kde
from scipy.stats import norm, chi2_contingency

from sklearn import preprocessing
#%config InlineBackend.figure_format = 'svg' для большей четкости графиков
matplotlib.style.use('ggplot')
%matplotlib inline

In [ ]:
#Удаление
del df['NR']

In [ ]:
#Вероятностная гистограмма 
df['SalePrice'].hist(density=True, bins=60)
#Сравнение гистограмм
df.groupby('Status')['Length'].plot.hist(alpha=.6)
plt.legend()

In [ ]:
#В случае очевидного смешения двух нормальных распределений, можно оценить их более подробно
df.groupby('Status')['Diagonal'].plot.hist(alpha=0.6)
df.groupby('MS Zoning')['SalePrice'].plot.hist(density=True) #Нормализованный вариант
plt.legend(loc='upper left')

In [ ]:
#Shapiro-Wilk позволяют проверить выборку на принадлежность к ГС и нормальность распредеелния
df = df.set_index(u'номер')
plt.hist(np.log10(df[u'население']), bins=50) #Применяем критерий Шапиро-Вилка после логарифмирования
res = stats.shapiro(np.log10(df[u'население']))
print('p-value: ', res[1]) #Если p маленькое, гипотезу о нормальности отвергаем

In [ ]:
#Работа с выбросами
town_2 = df.iloc[2:1004] #Обрезать аномалии

x = np.log10(df[u'население']) #Логарифмировать переменную (для лог-нормального распределения)
pd.Series(x).hist(bins=45)

exclude = int(len(df)/100*2.5) #Усеченное среднее (Среднее -2,5% самых малых и -2,5% наибольших)
redacted_town = df[exclude:len(df)-exclude]

In [ ]:
#Матрица диаграмм рассеивания: комплексное сравнение по нескольким переменным. Диагональ - ядерная оценка плотности
colors = {'genuine': 'green', 'counterfeit': 'red'}
scatter_matrix(df,               
               figsize=(6, 6), #размер картинки
               diagonal='kde', #плотность вместо гистограммы на диагонали
               c=df['Status'].replace(colors),  #цвета классов
               alpha=0.2 #степень прозрачности точек
              )

In [ ]:
#Корреляция двух переменных
plt.scatter(df['PRICE'], df['SQFT'])

In [ ]:
#Исправление опечаток
dataframe['gender'].map({'m': 'male', fem.': 'female', ...})
re.sub (r "\ ^ m \ $", ' Male ', ' male ', flags = re. IGNORECASE )

In [ ]:
#Недостающие значения. Drop или Impute
#Заполнить проблеы средним значением
rand = np.random.randint(average_age - 2*std_age, average_age + 2*std_age, size = count_nan_age)
dataframe["age"][np.isnan(dataframe["age"])] = rand
#Заполнить пробелы линейной регрессией (если нет выбросов)
#Hot-deck взять схожие переменных из другой БД
